In [1]:
import pandas as pd
import numpy as np
import autogluon.tabular as agt

In [2]:
train = pd.read_parquet('/home/xbar19/ML/vseros_geo/train.parquet')
geo_data = pd.read_parquet('vseros_geo/geo_data.parquet')
test = pd.read_parquet('vseros_geo/test.parquet')
sub = pd.read_csv('vseros_geo/sample_submit.csv')

In [12]:
len(train)

287522

In [3]:
geo_data

h3_index        lat         lon  osm_accommodation_closest  \
0       89013400d37ffff  69.507989   31.316197                        NaN   
1       89013404603ffff  69.546583   31.212575                        NaN   
2       89013404607ffff  69.548725   31.206807                        NaN   
3       8901340460fffff  69.549060   31.214871                        NaN   
4       89013414403ffff  69.422945   31.044869                        NaN   
...                 ...        ...         ...                        ...   
207517  89316ebab93ffff  42.865151  131.385408                        NaN   
207518  89316ebab9bffff  42.863995  131.382162                        NaN   
207519  89316ebabb3ffff  42.858614  131.390450                        NaN   
207520  89316ebabbbffff  42.857458  131.387205                        NaN   
207521  89316ebacd7ffff  42.904134  131.385675                        NaN   

        osm_accommodation_closest_5_mean  osm_accommodation_count_100  \
0                                    NaN                            0   
1                                    NaN                            0   
2                                    NaN                            0   
3                                    NaN                            0   
4                                    NaN                            0   
...                                  ...                          ...   
207517                               NaN                            0   
207518                               NaN                            0   
207519                               NaN                            0   
207520                               NaN                            0   
207521                               NaN                            0   

        osm_accommodation_count_1000  osm_accommodation_count_2000  \
0                                  0                             0   
1                                  0                             0   
2                                  0                             0   
3                                  0                             0   
4                                  0                             0   
...                              ...                           ...   
207517                             0                             0   
207518                             0                             0   
207519                             0                             0   
207520                             0                             0   
207521                             0                             0   

        osm_accommodation_count_500  osm_adult_closest  ...  \
0                                 0                NaN  ...   
1                                 0       19908.616467  ...   
2                                 0       19902.769554  ...   
3                                 0                NaN  ...   
4                                 0        8277.769193  ...   
...                             ...                ...  ...   
207517                            0                NaN  ...   
207518                            0                NaN  ...   
207519                            0                NaN  ...   
207520                            0                NaN  ...   
207521                            0                NaN  ...   

        osm_tourism_hotel_count_100  osm_tourism_hotel_count_1000  \
0                                 0                             0   
1                                 0                             0   
2                                 0                             0   
3                                 0                             0   
4                                 0                             0   
...                             ...                           ...   
207517                            0                             2   
207518                            0                            

In [3]:
train = train.merge(geo_data, on='h3_index', how='left')

In [15]:
len(test)

95841

In [4]:
test = test.merge(geo_data, on='h3_index', how='left')

In [5]:
del geo_data

In [18]:
test

h3_index industry_type  objects_count        lat         lon  \
0      892d5035853ffff         other              1  44.414100   41.510629   
1      89212d8314bffff         other              7  54.448932   61.271869   
2      8914d641e67ffff         other             57  48.493462  135.068027   
3      892d49d36a7ffff         other              1  49.055930   44.809794   
4      892d5e4856bffff         other              6  44.112895   43.489165   
...                ...           ...            ...        ...         ...   
95836  8910814768bffff        apteka              1  53.206768   50.206302   
95837  8911aa7ad2fffff  stomatologia              2  55.788150   37.598684   
95838  8911a326277ffff         other              1  55.560329   35.004892   
95839  890b2218e5bffff     zhivotnye              1  60.971466   69.040635   
95840  8910a0c6a6bffff         other              1  54.298597   44.485888   

       osm_accommodation_closest  osm_accommodation_closest_5_mean  \
0                            NaN                               NaN   
1                            NaN                               NaN   
2                            NaN                               NaN   
3                            NaN                               NaN   
4                            NaN                               NaN   
...                          ...                               ...   
95836                        NaN                               NaN   
95837                        NaN                               NaN   
95838                        NaN                               NaN   
95839                        NaN                               NaN   
95840                        NaN                               NaN   

       osm_accommodation_count_100  osm_accommodation_count_1000  \
0                                0                             0   
1                                0                             0   
2                                0                             0   
3                                0                             0   
4                                0                             0   
...                            ...                           ...   
95836                            0                             0   
95837                            0                             0   
95838                            0                             0   
95839                            0                             0   
95840                            0                             0   

       osm_accommodation_count_2000  ...  osm_tourism_hotel_count_100  \
0                                 0  ...                            0   
1                                 0  ...                            0   
2                                 0  ...                            0   
3                                 0  ...                            0   
4                                 0  ...                            0   
...                             ...  ...                          ...   
95836                             0  ...                            0   
95837                             0  ...                            0   
95838                             0  ...                            0   
95839                             0  ...                            0   
95840                             0  ...                            0   

       osm_tourism_hotel_count_1000  osm_tourism_hotel_count_2000  \
0                                 0                             0   
1                                 2                             2   
2                                 6                            16   
3                                10                            15   
4                                 0                             0   
...                             ...                           ...   
95836                             1                        

In [23]:
set(train.columns) - set(test.columns)

{'customer_count', 'transaction_count', 'transaction_sum'}

In [6]:
del train['customer_count'], train['transaction_count']

In [7]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

In [8]:
def haversine(lat1, lon1, lat2, lon2, r=6371):
    """
    Вычисляет расстояние между двумя точками на Земле (в км).
    Использует формулу Хаверсина.
    
    Параметры:
        lat1, lon1: широта и долгота первой точки (град.)
        lat2, lon2: широта и долгота второй точки (град.)
        r: радиус Земли (км)
    """
    # Переводим градусы в радианы
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    # Вычисляем a и c по формуле Хаверсина
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return r * c

# Координаты городов (широта, долгота)
CITIES = {
    "Moscow": (55.7558, 37.6176),
    "Saint_Petersburg": (59.9343, 30.3351),
    "Kyiv": (50.4501, 30.5234),
    "Minsk": (53.9045, 27.5615),
    "Beijing": (39.9042, 116.4074),
}

# Пример: добавляем расстояния до каждого города в датафрейм
for city, (city_lat, city_lon) in CITIES.items():
    train[f"distance_to_{city}"] = train.apply(
        lambda row: haversine(row["lat"], row["lon"], city_lat, city_lon), 
        axis=1
    )
    test[f"distance_to_{city}"] = test.apply(
        lambda row: haversine(row["lat"], row["lon"], city_lat, city_lon), 
        axis=1
    )

/tmp/ipykernel_363871/634870308.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train[f"distance_to_{city}"] = train.apply(
/tmp/ipykernel_363871/634870308.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train[f"distance_to_{city}"] = train.apply(
/tmp/ipykernel_363871/634870308.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

In [9]:
train

h3_index  industry_type  objects_count  transaction_sum  \
0       892d4499287ffff          other             11       4526297.17   
1       892d292496fffff          other             25       1513250.69   
2       8910b14981bffff  napitki_tabak              1        141914.00   
3       8910d9b4a77ffff    beauty_care              2        406543.00   
4       8911060520fffff          other             15      23526497.97   
...                 ...            ...            ...              ...   
287517  89119ec14cbffff    beauty_care              1          6350.00   
287518  8911a8d5907ffff    beauty_care              1         27066.00   
287519  892c604333bffff         apteka              3       1678931.96   
287520  89110eb928fffff          other              2         21026.50   
287521  8910e975927ffff          other              1           176.00   

              lat        lon  osm_accommodation_closest  \
0       47.216885  38.866084                        NaN   
1       45.421913  35.808481                        NaN   
2       51.611589  46.017180                        NaN   
3       57.144807  65.546458                        NaN   
4       59.846357  30.485972                        NaN   
...           ...        ...                        ...   
287517  51.705821  39.186322                        NaN   
287518  56.863041  35.877386                        NaN   
287519  46.318805  48.020249                        NaN   
287520  61.817248  34.084848                        NaN   
287521  61.385319  63.577489                        NaN   

        osm_accommodation_closest_5_mean  osm_accommodation_count_100  \
0                                    NaN                            0   
1                                    NaN                            0   
2                                    NaN                            0   
3                                    NaN                            0   
4                                    NaN                            0   
...                                  ...                          ...   
287517                               NaN                            0   
287518                               NaN                            0   
287519                               NaN                            0   
287520                               NaN                            0   
287521                               NaN                            0   

        osm_accommodation_count_1000  ...  osm_transportation_closest_5_mean  \
0                                  0  ...                         402.572417   
1                                  0  ...                        1008.037299   
2                                  0  ...                         377.527784   
3                                  0  ...                          92.412664   
4                                  0  ...                          75.765146   
...                              ...  ...                                ...   
287517                             0  ...                          90.791138   
287518                             0  ...                          24.489833   
287519                             0  ...                         121.922913   
287520                             0  ...                         382.984099   
287521                             0  ...                         444.323652   

        osm_transportation_count_100  osm_transportation_count_1000  \
0                                  0                             49   
1                                  0                              2   
2                                  0                             50   
3                                  2                            272   
4                                  3                             95   
...                              ...                            ...   
287517                             2                            136   
287518     

In [34]:
dist_cols = [x for x in train.columns if x.startswith('distance_to_')]

In [35]:
dist_cols

['distance_to_Moscow',
 'distance_to_Saint_Petersburg',
 'distance_to_Kyiv',
 'distance_to_Minsk',
 'distance_to_Beijing']

In [36]:
def is_near_capital(x):
    for col in x.index:
        if x[col] < 100:
            return 1
    return 0

In [37]:
train['is_near_capital'] = train[dist_cols].apply(is_near_capital, 1)
test['is_near_capital'] = test[dist_cols].apply(is_near_capital, 1)

/tmp/ipykernel_363871/1942037313.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['is_near_capital'] = train[dist_cols].apply(is_near_capital, 1)


In [10]:
target = 'transaction_sum'
X_train, X_val, y_train, y_val = train_test_split(train.drop(columns=target), train[target], test_size=0.2, random_state=19)

In [11]:
y_train = np.log10(y_train)
y_val = np.log10(y_val)

In [12]:
cat_cols = train.select_dtypes(object).columns.values

In [41]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

cv = KFold(n_splits=5)
target = 'transaction_sum'
scores = []
models = []

for i, (tr_idx, val_idx) in enumerate(cv.split(train)):
    X_train, X_val = train.drop(columns=target).iloc[tr_idx], train.drop(columns=target).iloc[val_idx]
    y_train, y_val = np.log10(train[target]).iloc[tr_idx], np.log10(train[target]).iloc[val_idx]
    model = CatBoostRegressor(eval_metric='MAE', loss_function='MAE', early_stopping_rounds=20, random_state=19, cat_features=cat_cols, verbose=100, iterations=5000, task_type='GPU')
    model.fit(X_train, y_train, eval_set=(X_val, y_val))
    val_preds = model.predict(X_val)
    score = mean_absolute_error(y_val, val_preds)
    print(f"Fold #{i}: {score}")
    scores.append(score)
    models.append(model)

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 0.8137937	test: 0.8121779	best: 0.8121779 (0)	total: 127ms	remaining: 10m 36s
100:	learn: 0.5675658	test: 0.5668198	best: 0.5668198 (100)	total: 3.08s	remaining: 2m 29s
200:	learn: 0.5375688	test: 0.5369671	best: 0.5369671 (200)	total: 5.68s	remaining: 2m 15s
300:	learn: 0.5292574	test: 0.5290303	best: 0.5290303 (300)	total: 8.15s	remaining: 2m 7s
400:	learn: 0.5244806	test: 0.5247226	best: 0.5247226 (400)	total: 10.7s	remaining: 2m 2s
500:	learn: 0.5210172	test: 0.5217832	best: 0.5217832 (500)	total: 13.1s	remaining: 1m 57s
600:	learn: 0.5182211	test: 0.5195375	best: 0.5195375 (600)	total: 15.5s	remaining: 1m 53s
700:	learn: 0.5159526	test: 0.5177713	best: 0.5177713 (700)	total: 17.9s	remaining: 1m 49s
800:	learn: 0.5141172	test: 0.5164549	best: 0.5164549 (800)	total: 20.3s	remaining: 1m 46s
900:	learn: 0.5124072	test: 0.5152904	best: 0.5152904 (900)	total: 22.8s	remaining: 1m 43s
1000:	learn: 0.5109508	test: 0.5143417	best: 0.5143417 (1000)	total: 25.3s	remaining: 1m 41s
11

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 0.8137426	test: 0.8124239	best: 0.8124239 (0)	total: 27.1ms	remaining: 2m 15s
100:	learn: 0.5675746	test: 0.5676358	best: 0.5676358 (100)	total: 2.82s	remaining: 2m 17s
200:	learn: 0.5374617	test: 0.5374839	best: 0.5374839 (200)	total: 5.39s	remaining: 2m 8s
300:	learn: 0.5290910	test: 0.5294039	best: 0.5294039 (300)	total: 7.95s	remaining: 2m 4s
400:	learn: 0.5244639	test: 0.5252366	best: 0.5252366 (400)	total: 10.5s	remaining: 2m
500:	learn: 0.5207535	test: 0.5220794	best: 0.5220794 (500)	total: 13s	remaining: 1m 56s
600:	learn: 0.5179965	test: 0.5198512	best: 0.5198512 (600)	total: 15.4s	remaining: 1m 52s
700:	learn: 0.5157841	test: 0.5181665	best: 0.5181665 (700)	total: 17.8s	remaining: 1m 49s
800:	learn: 0.5139966	test: 0.5169348	best: 0.5169348 (800)	total: 20.2s	remaining: 1m 45s
900:	learn: 0.5122666	test: 0.5157088	best: 0.5157088 (900)	total: 22.6s	remaining: 1m 42s
1000:	learn: 0.5109346	test: 0.5149263	best: 0.5149263 (1000)	total: 25s	remaining: 1m 40s
1100:	lear

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 0.8129120	test: 0.8156687	best: 0.8156687 (0)	total: 30.7ms	remaining: 2m 33s
100:	learn: 0.5672931	test: 0.5686097	best: 0.5686097 (100)	total: 3.35s	remaining: 2m 42s
200:	learn: 0.5372388	test: 0.5386955	best: 0.5386955 (200)	total: 6.29s	remaining: 2m 30s
300:	learn: 0.5287340	test: 0.5301849	best: 0.5301849 (300)	total: 8.96s	remaining: 2m 19s
400:	learn: 0.5239752	test: 0.5257794	best: 0.5257794 (400)	total: 11.5s	remaining: 2m 12s
500:	learn: 0.5203827	test: 0.5226870	best: 0.5226870 (500)	total: 14.3s	remaining: 2m 8s
600:	learn: 0.5175253	test: 0.5204050	best: 0.5204050 (600)	total: 16.8s	remaining: 2m 3s
700:	learn: 0.5155075	test: 0.5189281	best: 0.5189281 (700)	total: 19.4s	remaining: 1m 59s
800:	learn: 0.5136569	test: 0.5176230	best: 0.5176230 (800)	total: 22s	remaining: 1m 55s
900:	learn: 0.5120707	test: 0.5165846	best: 0.5165846 (900)	total: 24.7s	remaining: 1m 52s
1000:	learn: 0.5106724	test: 0.5156896	best: 0.5156896 (1000)	total: 27.3s	remaining: 1m 48s
1100

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 0.8134143	test: 0.8137280	best: 0.8137280 (0)	total: 38.2ms	remaining: 3m 10s
100:	learn: 0.5677968	test: 0.5677691	best: 0.5677691 (100)	total: 3.23s	remaining: 2m 36s
200:	learn: 0.5375455	test: 0.5376079	best: 0.5376079 (200)	total: 6.31s	remaining: 2m 30s
300:	learn: 0.5291937	test: 0.5291751	best: 0.5291751 (300)	total: 9.32s	remaining: 2m 25s
400:	learn: 0.5244967	test: 0.5245744	best: 0.5245744 (400)	total: 12.2s	remaining: 2m 19s
500:	learn: 0.5209859	test: 0.5214047	best: 0.5214047 (500)	total: 15s	remaining: 2m 14s
600:	learn: 0.5183123	test: 0.5190774	best: 0.5190774 (600)	total: 17.8s	remaining: 2m 10s
700:	learn: 0.5160592	test: 0.5173034	best: 0.5173034 (700)	total: 20.6s	remaining: 2m 6s
800:	learn: 0.5142098	test: 0.5158643	best: 0.5158643 (800)	total: 23.3s	remaining: 2m 2s
900:	learn: 0.5126011	test: 0.5147324	best: 0.5147324 (900)	total: 26.1s	remaining: 1m 58s
1000:	learn: 0.5111108	test: 0.5136755	best: 0.5136755 (1000)	total: 28.8s	remaining: 1m 54s
1100

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 0.8135431	test: 0.8133966	best: 0.8133966 (0)	total: 42.1ms	remaining: 3m 30s
100:	learn: 0.5675658	test: 0.5679631	best: 0.5679631 (100)	total: 3.01s	remaining: 2m 26s
200:	learn: 0.5371059	test: 0.5376010	best: 0.5376010 (200)	total: 5.9s	remaining: 2m 20s
300:	learn: 0.5288433	test: 0.5291446	best: 0.5291446 (300)	total: 8.71s	remaining: 2m 15s
400:	learn: 0.5242570	test: 0.5247203	best: 0.5247203 (400)	total: 11.5s	remaining: 2m 11s
500:	learn: 0.5205763	test: 0.5214168	best: 0.5214168 (500)	total: 14.3s	remaining: 2m 7s
600:	learn: 0.5179192	test: 0.5191947	best: 0.5191947 (600)	total: 17.1s	remaining: 2m 4s
700:	learn: 0.5157167	test: 0.5174960	best: 0.5174960 (700)	total: 19.9s	remaining: 2m 1s
800:	learn: 0.5139398	test: 0.5161926	best: 0.5161926 (800)	total: 22.6s	remaining: 1m 58s
900:	learn: 0.5123365	test: 0.5151182	best: 0.5151182 (900)	total: 25.3s	remaining: 1m 55s
1000:	learn: 0.5109696	test: 0.5142119	best: 0.5142119 (1000)	total: 28.1s	remaining: 1m 52s
1100

In [22]:
model = CatBoostRegressor(eval_metric='MAE', loss_function='MAE', early_stopping_rounds=20, random_state=19, cat_features=cat_cols, verbose=25, task_type='GPU', iterations=5000)
model.fit(X_train, y_train, eval_set=(X_val, y_val))

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 0.8131883	test: 0.8150664	best: 0.8150664 (0)	total: 32ms	remaining: 2m 39s
25:	learn: 0.7055559	test: 0.7054304	best: 0.7054304 (25)	total: 699ms	remaining: 2m 13s
50:	learn: 0.6357643	test: 0.6339544	best: 0.6339544 (50)	total: 1.38s	remaining: 2m 14s
75:	learn: 0.5933447	test: 0.5908179	best: 0.5908179 (75)	total: 2.04s	remaining: 2m 12s
100:	learn: 0.5687990	test: 0.5657052	best: 0.5657052 (100)	total: 2.69s	remaining: 2m 10s
125:	learn: 0.5548665	test: 0.5514186	best: 0.5514186 (125)	total: 3.37s	remaining: 2m 10s
150:	learn: 0.5469182	test: 0.5431958	best: 0.5431958 (150)	total: 4.01s	remaining: 2m 8s
175:	learn: 0.5418491	test: 0.5379485	best: 0.5379485 (175)	total: 4.66s	remaining: 2m 7s
200:	learn: 0.5382889	test: 0.5343195	best: 0.5343195 (200)	total: 5.33s	remaining: 2m 7s
225:	learn: 0.5354649	test: 0.5314066	best: 0.5314066 (225)	total: 5.94s	remaining: 2m 5s
250:	learn: 0.5332627	test: 0.5291395	best: 0.5291395 (250)	total: 6.58s	remaining: 2m 4s
275:	learn: 0.5

In [50]:
import autogluon.tabular as agt

In [9]:
target = 'transaction_sum'
train['log_' + target] = np.log10(train[target])

/tmp/ipykernel_114459/3940205201.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['log_' + target] = np.log10(train[target])


In [ ]:
predictor = agt.TabularPredictor(eval_metric='mean_absolute_error', label='log_' + target).fit(train.drop(columns=target), presets='best')

In [42]:
preds0 = []
for m in models:
    preds0.append(m.predict(test))

In [45]:
preds = []
for p in zip(*preds0):
    preds.append(np.median(p))

In [38]:
len(test)

95841

In [46]:
len(preds)

95841

In [43]:
preds = np.mean(preds0, 0)

In [25]:
preds = model.predict(test)

In [12]:
predictor.predict(test.iloc[:2])

0    4.470705
1    6.415885
Name: log_transaction_sum, dtype: float32

In [13]:
len(test)

95841

In [24]:
pd.DataFrame({'feature': model.feature_names_, 'score': model.feature_importances_})

feature      score
0                        h3_index   0.564697
1                   industry_type   9.270224
2                   objects_count  28.345138
3                             lat   3.698824
4                             lon   1.215844
..                            ...        ...
341            distance_to_Moscow   1.663553
342  distance_to_Saint_Petersburg   2.798881
343              distance_to_Kyiv   0.900859
344             distance_to_Minsk   1.364633
345           distance_to_Beijing   0.783857

[346 rows x 2 columns]

In [15]:
preds = predictor.predict(test, as_pandas=False)

In [18]:
sub0 = pd.read_csv('vseros_geo/sub_ctb_all_features_base_cv.csv')

In [19]:
sub1 = sub.copy()

In [20]:
sub['transaction_sum'] = (sub1['transaction_sum'] + sub0['transaction_sum']) / 2

In [44]:
sub['transaction_sum'] = 10**preds

In [45]:
sub['transaction_sum'].to_csv('vseros_geo/sub_dist_5000it_cv_is_cao.csv', index=True)